[View in Colaboratory](https://colab.research.google.com/github/ameasure/try_git/blob/master/Finetune.ipynb)

Transfer Learning with Finetune 

https://https://github.com/IndicoDataSolutions/finetune

In [2]:
!pip install finetune git+git://github.com/IndicoDataSolutions/finetune.git@92b28ecc1db0e33838995ea792b5d3b6f3dd7686


  Cloning git://github.com/IndicoDataSolutions/finetune.git (to revision 92b28ecc1db0e33838995ea792b5d3b6f3dd7686) to /tmp/pip-req-build-xz96x73i
    100% |████████████████████████████████| 51kB 4.1MB/s 
    100% |████████████████████████████████| 8.9MB 856kB/s 
    100% |████████████████████████████████| 31.2MB 1.5MB/s 
    100% |████████████████████████████████| 61kB 23.2MB/s 
    100% |████████████████████████████████| 143kB 15.2MB/s 
    100% |████████████████████████████████| 153kB 26.3MB/s 
  Running setup.py bdist_wheel for finetune ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8e/8d/9d/3fc6353867db3b2f034897211c30d2dcb13ed46e724447d20d
  Running setup.py bdist_wheel for IndicoIo ... - \ done
  Stored in directory: /root/.cache/pip/wheels/92/45/b8/3cc5048729333ee7fafff88522c9537d999b176453ca127798
  Running setup.py bdist_wheel for ftfy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/e6/42/b1/0d7f9445a9df8920548ab9bb953d6587067c34bc47e5be962

In [10]:
!python -m spacy download en_core_web_lg
!pip install tables

    100% |████████████████████████████████| 852.3MB 28.1MB/s 
  Running setup.py install for en-core-web-lg ... - \ | / - \ | / - \ | / done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [12]:
!python -m spacy link en_core_web_lg en


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from finetune import Classifier

# read in our training data
df_train = pd.read_hdf('msha_2010-2011.h5')
# read in our validation data
df_valid = pd.read_hdf('msha_2012.h5')

In [14]:
# max_length is the maximum number of words we will use from each narrative
# behind the scenes finetune is setting aside 5% of the training data for validation
model = Classifier(verbose=False, batch_size=32, max_length=90, n_epochs=3)
model.fit(df_train['NARRATIVE'], df_train['INJ_BODY_PART'])

/usr/local/lib/python3.6/dist-packages/finetune/encoding.py:294: UserWarning: Some examples are longer than the max_length. Please trim documents or increase `max_length`. Fallback behaviour is to use the first 88 byte-pair encoded tokens
  "Fallback behaviour is to use the first {} byte-pair encoded tokens".format(max_length - 2)
/usr/local/lib/python3.6/dist-packages/finetune/encoding.py:233: UserWarning: Document is longer than max length allowed, trimming document to 90 tokens.
  max_length


Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `tf.variables_initializer` instead.
Train loss: 2.0182775372434465	 Validation loss: 1.3094253501305744
Train loss: 1.1533883723034464	 Validation loss: 1.181547429465339
Train loss: 0.8609702222153539	 Validation loss: 1.0683836955208326
Train loss: 0.7979224470055117	 Validation loss: 0.9787794041103615
Train loss: 0.7181133702163386	 Validation loss: 0.9104212771091338
Train loss: 0.621290335026025	 Validation loss: 0.8543589639592833
Train loss: 0.6094968880238256	 Validation loss: 0.812085539283956
Train loss: 0.5657256909892676	 Validation loss: 0.7716388466805049
Train loss: 0.49701036299197465	 Validation loss: 0.7435971454847637
Train loss: 0.45989694582949475	 Validation loss: 0.7229460965917613
Train loss: 0.47935538712566217	 Validation loss: 0.7081637357744024


In [23]:
# generate preditions
df_valid['PREDICTED_PART'] = model.predict(df_valid['NARRATIVE'].values)
# look at a sample
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART']].sample(5)

,NARRATIVE,INJ_BODY_PART,PREDICTED_PART
24755,The EE was dragging water hose up the slope when he slipped and fell twisting his right knee. The EE was taken off work on 2/18/12 due to tear found in his knee.,KNEE/PATELLA,KNEE/PATELLA
5720,The man was pushing up on a panel on a Kennedy wall when he felt a pain in his wrist.,WRIST,WRIST
14880,Loud noise / Hearing loss.,EAR(S) INTERNAL & HEARING,EAR(S) INTERNAL & HEARING
23610,Employee was sliding an electric motor from a loader bucket onto a truck bed when he pinched his right index finger between the motor and the truck bed causing a laceration.,FINGER(S)/THUMB,FINGER(S)/THUMB
18036,Standard Threshold Shift in employee's hearing in the right ear.,EAR(S) INTERNAL & HEARING,EAR(S) INTERNAL & HEARING


In [22]:
pd.options.display.max_colwidth=500

df_valid['PROB_DICT'] = model.predict_proba(df_valid['NARRATIVE'].values)
df_valid[['NARRATIVE', 'PREDICTED_PART', 'PROB_DICT']].head()

KeyError: ignored

In [0]:
# function that takes a row of our dataframe and returns the predicted probability
def get_probability(row):
    predicted_part = row['PREDICTED_PART']
    probability_dict = row['PROB_DICT']
    return probability_dict[predicted_part]

# apply get_probability to each row in our dataframe and store the result
df_valid['PREDICTED_PROB'] = df_valid.apply(func=get_probability, axis=1)
# take a peak at what we get
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART', 'PREDICTED_PROB']].sample(5).head()

In [24]:
from sklearn.metrics import accuracy_score, f1_score

mf1 = f1_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'], average='macro')
acc = accuracy_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'])
print('macro-f1:', mf1)
print('accuracy:', acc)

macro-f1: 0.6557191591602848
accuracy: 0.8256200177147919


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
